# Script to Gather and Merge Cozie Data
- Requires `credentials.py` to access our backend. Ask Matias or PJ for this file and place it in the same folder as this file
- Note that all data is saved as a `20191027.csv` so you do not need to run this to analyse the data

In [87]:
import requests
import numpy as np
import pandas as pd
from influxdb import InfluxDBClient, DataFrameClient
import datetime
import time
import json
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from mpl_toolkits import mplot3d
from shapely.geometry import Polygon, Point
%matplotlib inline
#User defined imports. if they are not in the folder then either make them or ask PJ
import credentials as cd
import space_names
import sensor_names
import geo_fence

geo_fence = geo_fence.geo_fence

spaces = space_names.space_names
sensors = sensor_names.sensor_names
client = DataFrameClient(cd.host, cd.port, cd.usr, cd.passwd, "SteerPath", ssl=True, verify_ssl=True)


### Functions to Query Data from Cozie, Steerpath, Sensing and Mbient.

- Cozie Data can be queried directly
- Steerpath, Sensing, and Mbient Data requires the credentials.py in order to execute a query from influxdb

In [93]:
def query_cozie(weeks):
	print("querying '{}' weeks of data from cozie".format(str(weeks)))
	payload = {'experiment-id': 'cresh', 'weeks': str(weeks)}
	response = requests.get('https://ay1bwnlt74.execute-api.us-east-1.amazonaws.com/test/request/', params = payload)



	response_data = json.loads(response.content)[1]['data']
	cozie_df = pd.read_json(response_data, orient='index')

	return(cozie_df)



def query_steerpath(weeks):
	print("querying '{}' weeks of data from Steerpath".format(str(weeks)))

		# Get current time and time x=52 weeks ago for querrzing influxdb
	to_time_str, from_time_str = get_current_time(weeks = weeks)

	# Get comfort data from learnign trail for the last 15min

	result = client.query("SELECT Longitude, Latitude, Floor, Space_id FROM people.autogen.Steerpath WHERE time > '{}' AND time < '{}' GROUP BY Userid".format(from_time_str,to_time_str))
	
	#print(result)
	# Create emtpy dataframe
	steerpath_df = pd.DataFrame()

	# Iterate through groups (rooms, and users)
	for key in result:
		# Get data frame belonging to a group
		current_df = result[key]
		# Set the location and user id to the data
		current_df["user_id"] =  key[1][0][1]
		#print(current_df)
		# Concat this sub dataframe to the main result data frame
		steerpath_df = pd.concat([steerpath_df, current_df], sort=False)


	# remove nulls
	#steerpath_df = steerpath_df[steerpath_df.Space_id != -1]
	return(steerpath_df)

def querySensing(weeks):
    print("querying '{}' weeks of data from Sensing".format(str(weeks)))
    
    to_time_str, from_time_str = get_current_time(weeks = weeks)
    print(to_time_str, from_time_str)
    
    result = client.query("SELECT temperature, humidity, noise, light, co2, voc FROM spaces.autogen.senSING WHERE time > '{}' AND time < '{}' GROUP BY room".format(from_time_str, to_time_str))

    sensing_df = pd.DataFrame()
    
    #Iterate through rooms
    for key in result:
        current_df = result[key]
        current_df["room"] = key[1][0][1]
        
        sensing_df = pd.concat([sensing_df, current_df], sort=False)
        #print(key[1][0][1])
    return(sensing_df)

def querySplitSensing(weeks):
    ## Influx is struggling with sensings large dataset so we split it into sepeerate queries
    print("querying '{}' weeks of data from Sensing as split times".format(str(weeks)))
    
    #current time, not sure if I should be using Sing time or now
    now = datetime.datetime.utcnow()
    #the time 1 year ago
    mid_time = now - datetime.timedelta(weeks=weeks/2)
    
    start_time = now - datetime.timedelta(weeks=weeks)

    #convert time to a format for querrying influxdb
    end_time_str = now.strftime('%Y-%m-%dT%H:%M:%SZ')
    mid_time_str = mid_time.strftime('%Y-%m-%dT%H:%M:%SZ')
    start_time_str = start_time.strftime('%Y-%m-%dT%H:%M:%SZ')

    print(end_time_str, mid_time_str, start_time_str)
    
    result = client.query("SELECT temperature, humidity, noise, light, co2, voc FROM spaces.autogen.senSING WHERE time > '{}' AND time < '{}' GROUP BY room".format(start_time_str, mid_time_str))
    result2 = client.query("SELECT temperature, humidity, noise, light, co2, voc FROM spaces.autogen.senSING WHERE time > '{}' AND time < '{}' GROUP BY room".format(mid_time_str, end_time_str))

    sensing_df = pd.DataFrame()
    
    #Iterate through rooms
    for key in result:
        current_df = result[key]
        current_df["room"] = key[1][0][1]
        
        sensing_df = pd.concat([sensing_df, current_df], sort=False)
        
    for key in result2:
        current_df = result2[key]
        current_df["room"] = key[1][0][1]
        
        sensing_df = pd.concat([sensing_df, current_df], sort=False)
        #print(key[1][0][1])
    return(sensing_df)

def queryMbient(weeks):
    print("querying '{}' weeks of data from Mbient".format(str(weeks)))
    
    to_time_str, from_time_str = get_current_time(weeks = weeks)
    
    result = client.query("SELECT temperature FROM people.autogen.mbient WHERE time > '{}' AND time < '{}' GROUP BY user_id".format(from_time_str, to_time_str))

    mbient_df = pd.DataFrame()
    
    #Iterate through rooms
    for key in result:
        current_df = result[key]
        current_df["user_id"] = key[1][0][1]
        
        mbient_df = pd.concat([mbient_df, current_df], sort=False)
        #print(key[1][0][1])
    return(mbient_df)


def get_current_time(weeks):
    #current time, not sure if I should be using Sing time or now
    now = datetime.datetime.utcnow()
    #the time 1 year ago
    time_ago = now - datetime.timedelta(weeks=weeks)

    #convert time to a format for querrying influxdb
    now_str = now.strftime('%Y-%m-%dT%H:%M:%SZ')
    from_time_str = time_ago.strftime('%Y-%m-%dT%H:%M:%SZ')

    #returning as global variable - TODO tidy this up to avoid future issues
    return(now_str, from_time_str)



### Functions to merge the data sets

- It is important that the dataframes are deep copied 

In [94]:
def merge_cozie_steerpath(cozie_df, steerpath_df):

    #making deep copies
    merged_cozie_df = cozie_df.copy(deep=True)
    merged_steerpath_df = steerpath_df.copy(deep=True)

    #Extract time index to column
    merged_cozie_df["time"] = merged_cozie_df.index.tz_localize('UTC') 
    merged_steerpath_df["time"] = merged_steerpath_df.index

    merged_cozie_df.sort_values(by=['time'], inplace=True)
    merged_steerpath_df.sort_values(by=['time'], inplace=True)
    
    #print("cozie df is:", merged_cozie_df)
    localised_cozie_df = pd.merge_asof(merged_cozie_df, merged_steerpath_df, on='time', by="user_id", tolerance=pd.Timedelta(seconds = 120), direction = "nearest")
    
    localised_cozie_df.set_index("time", inplace=True)
    #print(localised_cozie_df)

    return(localised_cozie_df)

def merge_cozie_sensing(raw_df, sensing_df):
    
    merged_cozie_df = raw_df.copy(deep=True)
    merged_sensing_df = sensing_df.copy(deep=True)
    
    #merged_cozie_df["time"] = merged_cozie_df.index
    merged_sensing_df["time"] = merged_sensing_df.index
    merged_cozie_df.reset_index(inplace=True)

    merged_cozie_df.sort_values(by=['time'], inplace=True)
    merged_sensing_df.sort_values(by=['time'], inplace=True)

    sensor_localised_cozie_df = pd.merge_asof(merged_cozie_df, merged_sensing_df, on='time', by="room", tolerance=pd.Timedelta(seconds = 120), direction = "nearest")
    
    return(sensor_localised_cozie_df)

def merge_cozie_mbient(raw_df, sensing_df):
    merged_cozie_df = raw_df.copy(deep=True)
    merged_mbient_df = sensing_df.copy(deep=True)
    
    merged_cozie_df.reset_index(inplace=True)
    merged_mbient_df["time"] = merged_mbient_df.index
    
    merged_cozie_df.sort_values(by=['time'], inplace=True)
    merged_mbient_df.sort_values(by=['time'], inplace=True)
    
    mbient_cozie_df = pd.merge_asof(merged_cozie_df, merged_mbient_df, on='time', by="user_id", tolerance=pd.Timedelta(seconds = 120), direction= "nearest")
    
    return(mbient_cozie_df)

In [95]:
def filter_values(dataframe):

	filter_df = dataframe.copy(deep=True)
	# Filter values based on time close to each other 

	
	filter_df = filter_df[filter_df['responseSpeed_cozie'] > 1.0]

	#Filter error user
	filter_df = filter_df[filter_df['user_id'] != "cresh04 "]

	filter_df.dropna(subset=['thermal_cozie'], inplace = True)
	#print(filter_df)

	return(filter_df)

In [96]:
### Functions to re organise the data frames in order to evaluate the total results

In [97]:
def cluster_total_cozie(dataframe, group, threshold):
    cluster_df = dataframe.copy(deep=True)
    # remapping
    cluster_df['Prefer Cooler'] = cluster_df['thermal_cozie'][cluster_df.thermal_cozie == 11.0] #Too Hot
    cluster_df['Prefer Warmer'] = cluster_df['thermal_cozie'][cluster_df.thermal_cozie == 9.0] # Too COld
    cluster_df['Thermaly Comfy'] = cluster_df['thermal_cozie'][cluster_df.thermal_cozie == 10.0] # COmfy
    
    cluster_df['Prefer Quieter'] = cluster_df['noise_cozie'][cluster_df.noise_cozie == 11.0] # Too Loud
    cluster_df['Prefer Louder'] = cluster_df['noise_cozie'][cluster_df.noise_cozie == 9.0] # Too Quiet
    cluster_df['Aurally Comfy'] = cluster_df['noise_cozie'][cluster_df.noise_cozie == 10.0] # Comfy

    cluster_df['Prefer Dimmer'] = cluster_df['light_cozie'][cluster_df.light_cozie == 11.0] # Too bright
    cluster_df['Prefer Brighter'] = cluster_df['light_cozie'][cluster_df.light_cozie == 9.0] # Too Dark
    cluster_df['Visually Comfy'] = cluster_df['light_cozie'][cluster_df.light_cozie == 10.0] # Comfy
    

    # group
    group_df=cluster_df.groupby(group).count()
    group_df = group_df[group_df.thermal >= threshold]

    group_df[['Prefer Cooler', 'Prefer Warmer', 'Thermaly Comfy']] = group_df[['Prefer Cooler', 'Prefer Warmer', 'Thermaly Comfy']].div(group_df.thermal, axis=0)
    group_df[['Prefer Quieter', 'Prefer Louder', 'Aurally Comfy']] = group_df[['Prefer Quieter', 'Prefer Louder', 'Aurally Comfy']].div(group_df.noise, axis=0)
    group_df[['Prefer Dimmer', 'Prefer Brighter', 'Visually Comfy']] = group_df[['Prefer Dimmer', 'Prefer Brighter', 'Visually Comfy']].div(group_df.light, axis=0) 

    group_df.drop(["thermal_cozie", "light_cozie", "noise_cozie", "comfort_cozie", "heartRate_cozie", "inOffice", "indoorOutdoor", "lat", "lon", "mood", "responseSpeed"], axis=1, inplace=True)

    #print(group_df)

    # Plot the normalised df
    ax = sns.clustermap(group_df, cmap="Blues", metric="euclidean", method="complete", annot=False, fmt='g', col_cluster=True)
    return (group_df)

### Actually Do Stuff

- This is the equivalent of the `main.py` here

In [98]:
# Setting data aquisition to 5 weeks
weeks = 8

# Query the data
cozie_df = query_cozie(weeks)
steerpath_df = query_steerpath(weeks)
mbient_df = queryMbient(weeks)
# Sensing Data is too large so it needs to be split into two seperate calls and merged
sensing_df = querySplitSensing(weeks) #querySensing(weeks)

print("done")

# Rename the colums for ease
mbient_df.rename(columns={"temperature": "temperature_mbient"}, inplace=True)

sensing_df.rename(columns = {"temperature":"temperature_sensing",
                            "noise":"noise_sensing",
                            "light":"light_sensing",
                            "co2" :"co2_sensing",
                            "voc" :"voc_sensing",
                            "humidity" :"humidity_sensing",
                            }, inplace=True)

cozie_df.drop(["inOffice", "indoorOutdoor", "mood"], axis = 1, inplace = True)

cozie_df.rename(columns = {"thermal":"thermal_cozie",
                            "noise":"noise_cozie",
                            "light":"light_cozie",
                            "lat" :"lat_cozie",
                            "lon" :"lon_cozie",
                            "comfort" :"comfort_cozie",
                            "responseSpeed" :"responseSpeed_cozie",
                           "heartRate": "heartRate_cozie"
                            }, inplace = True)


querying '8' weeks of data from cozie
querying '8' weeks of data from Steerpath
querying '8' weeks of data from Mbient
querying '8' weeks of data from Sensing as split times
2019-11-15T07:23:58Z 2019-10-18T07:23:58Z 2019-09-20T07:23:58Z
done


In [99]:

#space_names
#localised_cozie_df.Space_id.map(spaces)
#sensing_df[sensing_df["room"]=="OPEN_EXHIBITION_SPACE_2"]

In [100]:
# Function to determine whether the data point is geofenced to a certain room
def locate_room(df):
    
    check_point = Point(df.Longitude, df.Latitude)
    for rooms in geo_fence:
        poly = Polygon(rooms["Coordinates"])
        if (check_point.within(poly)) and rooms["Floor"] == df.Floor:
            current_space = rooms["SpaceName"]
            return(current_space)
    # No room found
    #return(np.NaN)
            

In [101]:
filtered_cozie_df = filter_values(cozie_df)
localised_cozie_df = merge_cozie_steerpath(cozie_df, steerpath_df)
localised_cozie_df["room"] = localised_cozie_df.apply(locate_room, axis=1)
#localised_cozie_df["steerpath_room"] = localised_cozie_df.Space_id.map(spaces)
#localised_cozie_df["room"] = localised_cozie_df.steerpath_room.map(sensors)
sensor_localised_cozie_df = merge_cozie_sensing(localised_cozie_df, sensing_df)

mbient_sensor_localised_cozie_df = merge_cozie_mbient(sensor_localised_cozie_df, mbient_df)

In [102]:
# Just running some checks
# sensor_localised_cozie_df[sensor_localised_cozie_df["room"]=="EXECUTIVE_SEMINAR_ROOM_A"]

In [103]:
csv_name = str(datetime.datetime.date(datetime.datetime.now())) + "_" + "cozie_full" + ".csv"
mbient_sensor_localised_cozie_df.to_csv(csv_name)

In [74]:
irc_library_df = localised_cozie_df[localised_cozie_df["Space_id"] == 20 ]

In [75]:
cluster_total_cozie(cozie_df, group='user_id', threshold = 5)
plt.show()

AttributeError: 'DataFrame' object has no attribute 'thermal'

In [ ]:
cluster_total_cozie(irc_library_df.drop(["time", "Space_id", "Floor", "Longitude", "Latitude"], axis=1), group='user_id', threshold = 0)
plt.show()

In [ ]:
cluster_total_cozie(localised_cozie_df.drop(["time", "user_id", "Floor", "Longitude", "Latitude"], axis=1), group='Space_id', threshold = 5)

In [22]:
old = pd.read_csv("20191025_cozie_full.csv").dropna(subset = ["temperature_sensing"])
new = pd.read_csv("20191027_cozie_full.csv").dropna(subset = ["temperature_sensing"])



In [23]:
new

,Unnamed: 0,index,time,comfort_cozie,heartRate_cozie,lat_cozie,light_cozie,lon_cozie,noise_cozie,responseSpeed_cozie,...,Longitude,Space_id,room,co2_sensing,humidity_sensing,light_sensing,noise_sensing,temperature_sensing,voc_sensing,temperature_mbient
8,8,8,2019-09-30 04:08:23.750000+00:00,10.0,68.0,NaN,10.0,NaN,10.0,3.573,...,103.771051,45.0,BUDS_LAB,732.0,50.8191,158.0,47.0,26.1212,66.0,NaN
10,10,10,2019-09-30 04:48:46.900000+00:00,9.0,78.0,NaN,10.0,NaN,10.0,7.741,...,103.770332,0.0,OPEN_EXHIBITION_SPACE_2,558.0,96.7441,27.0,69.0,24.1869,50.0,NaN
14,14,14,2019-09-30 05:38:46.872000+00:00,NaN,62.0,NaN,10.0,NaN,10.0,1499.477,...,103.771047,45.0,BUDS_LAB,674.0,55.0346,200.0,47.0,23.0508,53.0,27.250
19,19,19,2019-09-30 08:53:55.188000+00:00,10.0,80.0,NaN,10.0,NaN,10.0,0.936,...,103.770454,2.0,ENERGY_LAB_1,534.0,69.6409,31.0,52.0,26.0001,133.0,28.875
20,20,20,2019-09-30 09:00:21.186000+00:00,9.0,70.0,NaN,10.0,NaN,10.0,2.593,...,103.770420,2.0,ENERGY_LAB_1,572.0,68.8295,32.0,47.0,26.0454,133.0,29.125
21,21,21,2019-09-30 09:03:45.390000+00:00,10.0,84.0,NaN,10.0,NaN,10.0,1.430,...,103.770454,2.0,ENERGY_LAB_1,580.0,68.6628,31.0,46.0,26.0728,130.0,28.750
23,23,23,2019-09-30 09:27:24.914000+00:00,NaN,74.0,NaN,10.0,NaN,10.0,1193.176,...,103.770364,0.0,OPEN_EXHIBITION_SPACE_2,504.0,90.0784,90.0,66.0,26.4108,50.0,28.500
25,25,25,2019-09-30 10:00:13.137000+00:00,NaN,65.0,NaN,10.0,NaN,10.0,2785.378,...,103.771034,45.0,BUILDING_MATERIALS_LAB,658.0,54.5723,60.0,47.0,23.5594,127.0,30.000
26,26,26,2019-09-30 10:07:57.632000+00:00,9.0,77.0,NaN,10.0,NaN,10.0,5.510,...,103.771029,45.0,BUILDING_MATERIALS_LAB,658.0,54.2316,64.0,44.0,23.5425,132.0,28.250
37,37,37,2019-10-01 02:20:36.578000+00:00,9.0,82.0,NaN,10.0,NaN,10.0,3.150,...,103.770423,-1.0,ENERGY_LAB_1,448.0,69.0254,51.0,50.0,25.9548,137.0,29.375
